# Task 1: Vanilla CNN

In the following, vanilla CNN for ECG heartbeat classification are trained. We trained different models from data from the [MIT-BIH Arrythmia Database](https://physionet.org/content/mitdb/1.0.0/) and [PTB Diagnostic ECG Database](https://physionet.org/physiobank/database/ptbdb/). 

First three different net-architectures are compared with a 5-fold cross validation in order to determine the best structure among these. We compare the architecture from the baseline (Baseline-Class) to a [VGG-net similar architecture](https://www.hindawi.com/journals/jhe/2021/7167891/) (DoubleConv-Class) and a alternating convolutional and maxpool layer architecture (VanillaCNN-Class).


Second, the hyperparameters are tuned with a grid search. Then the final model is trained with all available training data and the classes for the test set are predicted. For the MIT-BIH data, the VanillaCNN-model is further evaluated and on the PTBDB data, the DoubleConv-model is further evaluated. 

Further information can be found in the corresponding section of the report.

In [ ]:
import numpy as np
import json
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, LearningRateScheduler, ReduceLROnPlateau
from keras import losses, activations, models
from tensorflow.keras import optimizers
import pandas as pd
from sklearn.model_selection import train_test_split, KFold
from sklearn.metrics import f1_score, accuracy_score, roc_auc_score, average_precision_score
import sys
sys.path.append("../")
from models import *

## MIT-BIH Arryhtmia Database

In [ ]:
# read data
df_train = pd.read_csv("../input/mitbih_train.csv", header=None)
df_train = df_train.sample(frac=1)
df_test = pd.read_csv("../input/mitbih_test.csv", header=None)

Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [ ]:
# Compare Different VanillaCNN structures
accs = {
    "basemodel" : [],
    "doubleconv_model" : [],
    "init_model" : []
}

for fold, (train, val) in enumerate(KFold(n_splits=5, shuffle=True,random_state = 123).split(X,Y)):
    
    # create callback lists for different models
    file_path_bl = f"Results/baseline_cnn_mitbih.h5"
    checkpoint_bl = ModelCheckpoint(file_path_bl, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_bl = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_bl = [checkpoint_bl, early_bl] 
    
    file_path_doubleconv = f"Results/doubleconv_cnn_mitbih.h5"
    checkpoint_doubleconv = ModelCheckpoint(file_path_doubleconv, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_doubleconv = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_doubleconv = [checkpoint_doubleconv, early_doubleconv] 
    
    file_path_init = f"Results/init_cnn_mitbih.h5"
    checkpoint_init = ModelCheckpoint(file_path_init, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_init = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_init = [checkpoint_init, early_init]
    
    baseline_model = Baseline(5, callbacks=callbacks_list_bl)
    doubleconv_model = DoubleConvCNN(5,0.1, optimizers.Adam(0.001),callbacks=callbacks_list_doubleconv)
    init_model = VanillaCNN(5,0.1, optimizers.Adam(0.001), callbacks=callbacks_list_init)
    
    # train models
    baseline_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=2,  validation_data = (X[val],Y[val]) )
    doubleconv_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=2, validation_data = (X[val],Y[val]) )
    init_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=2, validation_data = (X[val],Y[val]) )
    
    # evaluate models
    accs["basemodel"].append(baseline_model.score(X[val],Y[val]))
    accs["doubleconv_model"].append(doubleconv_model.score(X[val],Y[val]))
    accs["init_model"].append(init_model.score(X[val],Y[val]))
    
    with open("Results/CNN_CV_results_MITBIH.json", "w") as outfile:
        json.dump(accs, outfile)

In [ ]:
f = open('Results/CNN_CV_results_MITBIH.json')
results_mitbih = json.load(f)
for model in results_mitbih.keys():
    print("Architecture", model)
    mean_acc = sum(elt for elt in results_mitbih[model])/len(results_mitbih[model])
    print(f"mean acc: {mean_acc}")

The CNN architecture from the basemodel is best performing, out of the three different architectures. However, we continue with the Conv-MaxPool model (second best performing model), because the basemodel might already be hyperparameter tuned and we assume that the objective of the VanillaCNN task is not to reproduce the basemodel. The Conv-MaxPool model obtained the following performances:

**mean loss: 0.06200777292251587** 	 

**mean acc: 0.9842497110366821**

### Hyperparametertuning
The following hyperparameters (dropout rate, learning_rate, optimizer, activation function, decay) are tuned with a 5-fold CV grid search. 

In [ ]:
grid_search_results = {}

dropout_rate = [0.1,0.3]
opts = ["Adam", "SGD", "rmsprop"]
learning_rates = [("const", 0.0001), ("const", 0.001), ("const",0.01), ("lr_schedule", 0.9), ("lr_schedule", 0.1), ("lr_schedule", 0.01)]

for dr in dropout_rate:
    for opt in opts: 
        for mode,factor in learning_rates:
            if mode == "const":
                lr = factor
            else: 
                lr = optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3,decay_steps=1000,decay_rate=factor)
            if opt== "Adam":
                optimizer = optimizers.Adam(learning_rate = lr)
            elif opt == "SGD":
                optimizer = optimizers.SGD(learning_rate = lr)
            elif opt == "rmsprop":
                optimizer = optimizers.RMSprop(learning_rate = lr)
            
            grid_search_results[f"{dr}_{opt}_{mode}_{factor}"] = []
            
            for fold, (train, val) in enumerate(KFold(n_splits=3, shuffle=True,random_state = 123).split(X,Y)):
                # create callback lists for different models
                file_path_vanillacnn = f"Results/init_cnn_mitbih_grid_search.h5"
                checkpoint_vanillacnn = ModelCheckpoint(file_path_init, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
                early_vanillacnn = EarlyStopping(monitor='val_acc', patience=7)
                callbacks_list_vanillacnn = [checkpoint_vanillacnn, early_vanillacnn]
                
                vanillacnn_model = VanillaCNN(5, dr, optimizer, callbacks=callbacks_list_vanillacnn)
                vanillacnn_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=2, validation_data = (X[val],Y[val]) )
    
                grid_search_results[f"{dr}_{opt}_{mode}_{factor}"].append(vanillacnn_model.score(X[val],Y[val]))

                with open("Results/CNN_Hyperparam_CV_results_MITBIH.json", "w") as outfile:
                    json.dump(grid_search_results, outfile)

In [ ]:
f = open('Results/CNN_Hyperparam_CV_results_MITBIH.json')
grid_search_results = json.load(f)
means = []
for k in grid_search_results.keys():
    mean_acc = sum(eltfor elt in grid_search_results[k])/len(grid_search_results[k])
    means.append((k, mean_acc))

means.sort(key=lambda a: (-a))
means[0]

Based on the grid search the optimal parameters for the model are: 

dropout rate: **0.1** 

optimizer: **Adam**

learning rate: **constant of 0.001***

These parameters obtained performances of (average loss,  average accuracy): 

**(0.054875085751215615, 0.9846951365470886)**

### Train Final Model

The final model with parameters set to the optimal values from the grid search is trained on all given training data.

In [ ]:
file_path_vanillacnn = f"Results/VanillaCNN_MITBIH.h5"
checkpoint_vanillacnn = ModelCheckpoint(file_path_vanillacnn, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
early_init_vanillacnn = EarlyStopping(monitor='val_acc', patience=7)
callbacks_list_vanillacnn = [checkpoint_vanillacnn, early_init_vanillacnn]
lr = 0.001
optimizer = optimizers.Adam(learning_rate = lr)
vanillacnn_model = VanillaCNN(5,0.1, optimizer, callbacks=callbacks_list_vanillacnn)
vanillacnn_model.fit(X, Y, epochs=200, batch_size=128, verbose=1, validation_split=0.1)

### Prediction

In [ ]:
vanillacnn_model.load_weights("Results/VanillaCNN_MITBIH.h5")
pred_test = vanillacnn_model.predict(X_test)

f1 = f1_score(Y_test, pred_test,average="macro")
print(f"Test f1 score : {f1} ")
acc = accuracy_score(Y_test, pred_test)
print(f"Test accuracy : {acc} ")

mean f1 0.9091180122085432, var f1: 1.0683676232951701e-05


mean acc 0.983957610085876, var acc: 6.00426077167262e-07

## PTB Diagonstic ECG Database

In [ ]:
# read data
df_1 = pd.read_csv("../input/ptbdb_normal.csv", header=None)
df_2 = pd.read_csv("../input/ptbdb_abnormal.csv", header=None)
df = pd.concat([df_1, df_2])

df_train, df_test = train_test_split(df, test_size=0.2, random_state=1337, stratify=df[187])


Y = np.array(df_train[187].values).astype(np.int8)
X = np.array(df_train[list(range(187))].values)[..., np.newaxis]

Y_test = np.array(df_test[187].values).astype(np.int8)
X_test = np.array(df_test[list(range(187))].values)[..., np.newaxis]

In [ ]:
accs = {
    "basemodel" : [],
    "doubleconv_model" : [],
    "init_model" : []
}

for fold, (train, val) in enumerate(KFold(n_splits=5, shuffle=True,random_state = 123).split(X,Y)):
    # create callback lists for different models
    file_path_bl = f"Results/baseline_cnn_ptbdb.h5"
    checkpoint_bl = ModelCheckpoint(file_path_bl, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_bl = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_bl = [checkpoint_bl, early_bl] 
    
    file_path_doubleconv = f"Results/doubleconv_cnn_ptbdb.h5"
    checkpoint_doubleconv = ModelCheckpoint(file_path_doubleconv, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_doubleconv = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_doubleconv = [checkpoint_doubleconv, early_doubleconv] 
    
    file_path_init = f"Results/init_cnn_ptbdb.h5"
    checkpoint_init = ModelCheckpoint(file_path_init, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
    early_init = EarlyStopping(monitor='val_acc', patience=7)
    callbacks_list_init = [checkpoint_init, early_init]
    
    baseline_model = Baseline(1, callbacks=callbacks_list_bl)
    doubleconv_model = DoubleConvCNN(1,0.1, optimizers.Adam(0.001), callbacks=callbacks_list_doubleconv)
    init_model = VanillaCNN(1,0.1, optimizers.Adam(0.001), callbacks=callbacks_list_init)
    
    # train models
    baseline_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=1, validation_data = (X[val],Y[val]) )
    doubleconv_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=1, validation_data = (X[val],Y[val]) )
    init_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=1, validation_data = (X[val],Y[val]) )
    
    # evaluate models
    accs["basemodel"].append(baseline_model.score(X[val],Y[val]))
    accs["doubleconv_model"].append(doubleconv_model.score(X[val],Y[val]))
    accs["init_model"].append(init_model.score(X[val],Y[val]))
    
    with open("Results/CNN_CV_results_PTBDB.json", "w") as outfile:
        json.dump(accs, outfile)

In [ ]:
f = open('Results/CNN_CV_results_PTBDB.json')
results_ptbdb = json.load(f)
means_ptbdb = []
for model in results_ptbdb.keys():
    print("Architecture", model)
    mean_acc = sum(elt for elt in results_ptbdb[model])/len(results_ptbdb[model])
    print(f"mean acc: {mean_acc}")

The CNN architecture from the basemodel is best performing, out of the three different architectures. However, we continue with the DoubleConv model (second best performing model), because the basemodel might already be hyperparameter tuned and we assume that the objective of the VanillaCNN task is not to reproduce the basemodel. The DoubleConv model obtained the following performances:

**mean loss: 0.05904214382171631** 	 

**mean acc: 0.9812728762626648**

### Hyperparameter Tuning

In [ ]:
grid_search_results = {}

dropout_rate = [0.1,0.3]

opts = ["Adam", "SGD", "rmsprop"]
learning_rates = [("const", 0.0001), ("const", 0.001), ("const",0.01), ("lr_schedule", 0.9), ("lr_schedule", 0.1), ("lr_schedule", 0.01)]

for dr in dropout_rate:
    for opt in opts: 
        for mode,factor in learning_rates:
            if mode == "const":
                lr = factor
            else: 
                lr = optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3,decay_steps=1000,decay_rate=factor)
            
            if opt== "Adam":
                optimizer = optimizers.Adam(learning_rate = lr)
            elif opt == "SGD":
                optimizer = optimizers.SGD(learning_rate = lr)
            elif opt == "rmsprop":
                optimizer = optimizers.RMSprop(learning_rate = lr)
            
            grid_search_results[f"{dr}_{opt}_{mode}_{factor}"] = []
            
            for fold, (train, val) in enumerate(KFold(n_splits=3, shuffle=True,random_state = 123).split(X,Y)):
                file_path_doubleconv = f"Results/doubleconv_cnn_ptbdb.h5"
                checkpoint_doubleconv = ModelCheckpoint(file_path_doubleconv, monitor='val_acc', verbose=1, save_best_only=True, mode='max')    
                early_doubleconv = EarlyStopping(monitor='val_acc', patience=7)
                callbacks_list_doubleconv = [checkpoint_doubleconv, early_doubleconv] 

                doubleconv_model = DoubleConvCNN(1,dr, optimizer)
                doubleconv_model.fit(X[train], Y[train], epochs=100, batch_size=128, verbose=1, callbacks=callbacks_list_doubleconv, validation_data = (X[val],Y[val]) )
    
                grid_search_results[f"{dr}_{opt}_{mode}_{factor}"].append(doubleconv_model.score(X[val],Y[val]))

                with open("Results/CNN_Hyperparam_CV_results_PTBDB.json", "w") as outfile:
                    json.dump(grid_search_results, outfile)

In [ ]:
f = open('Results/CNN_Hyperparam_CV_results_PTBDB.json')
grid_search_results = json.load(f)
means = []
for k in grid_search_results.keys():
    mean_acc = sum(elt for elt in grid_search_results[k])/len(grid_search_results[k])
    means.append((k, mean_acc))

means.sort(key=lambda a: (-a))
means[0]

Based on the grid search the optimal parameters for the model are: 

dropout rate: **0.1** 

optimizer: **Adam**

learning rate: **decay with decay rate of 0.9***

These parameters obtained performances of (average loss,  average accuracy): 

**(0.056583555042743684, 0.9809292674064636)**


### Train Final Model

In [ ]:
file_path_doubleconv = f"Results/VanillaCNN_PTBDB.h5"
checkpoint_doubleconv = ModelCheckpoint(file_path_doubleconv, monitor='val_acc', verbose=2, save_best_only=True, mode='max')    
early_doubleconv = EarlyStopping(monitor='val_acc', patience=7)
callbacks_list_doubleconv = [checkpoint_doubleconv, early_doubleconv] 

lr = optimizers.schedules.ExponentialDecay(initial_learning_rate=1e-3,decay_steps=1000,decay_rate=0.9)
optimizer = optimizers.Adam(learning_rate = lr)
doubleconv_model = DoubleConvCNN(1,0.1, optimizer,callbacks=callbacks_list_doubleconv)

doubleconv_model.fit(X, Y, epochs=100, batch_size=128, verbose=1,  validation_split=0.1)

## Prediction

In [ ]:
doubleconv_model.load_weights("Results/VanillaCNN_PTBDB.h5")
pred_test = doubleconv_model.predict(X_test)
#pred_test = (pred_test>0.5).astype(np.int8)

f1 = f1_score(Y_test, pred_test)

print("Test f1 score : %s "% f1)

acc = accuracy_score(Y_test, pred_test)

print("Test accuracy : %s "% acc)

auroc = roc_auc_score(Y_test, pred_test)

print("Test AUROC : %s "% auroc)

auprc = average_precision_score(Y_test, pred_test)

print("Test AUPRC : %s "% auprc)